---
title: "Proyecto deteccion de fraude en tarjeta de credito"
author: "Leandro Soto Miranda"
date: "2024-10-24"
format: 
  html: 
    toc: true 
    code-fold: true
---


## 1. Definir problema

- En este proyecto, analizaremos un conjunto de datos relacionaciodnado con la productividad

## 2. Recopilación de datos


In [ ]:
# Importación de librerías
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

    
# Carga de datos
df = pd.read_csv('data.csv', sep=",")

# Resumen estadístico
# Mostrar las primeras filas del dataset
print("Primeras filas del dataset:")
print(df.head())

# Mostrar información general del dataset
print("\nInformación del dataset:")
print(df.info())

# Descripción estadística básica del dataset
print("\nDescripción estadística:")
print(df.describe())

info = df.shape
print("\nLa cantidad de filas y columnas en nuestro dataframe es de:",info)

tipos = df.dtypes
print("\nTipos de datos presentes en el dataset:\n",tipos)

## 3. Análisis de datos por Variable
## Análisis de datos cuantitativos


In [ ]:
# Función para el análisis univariado de variables cuantitativas
def analizar_variable_cuantitativa(df, columna):
    mean = np.mean(df[columna])
    median = np.median(df[columna])
    std = np.std(df[columna])
    min_value = np.min(df[columna])
    max_value = np.max(df[columna])

    print(f"Análisis de la Variable '{columna}'")
    print(f"Media: {mean:.2f}")
    print(f"Mediana: {median:.2f}")
    print(f"Desviación Estándar: {std:.2f}")
    print(f"Valor Mínimo: {min_value:.2f}")
    print(f"Valor Máximo: {max_value:.2f}")
    print("\n")

    # Visualización de la distribución (Histograma con KDE)
    plt.figure(figsize=(10, 6))
    sns.histplot(df[columna], bins=30, kde=True, color='#4C72B0')
    plt.title(f'Distribución de {columna}')
    plt.xlabel(columna)
    plt.ylabel('Frecuencia')
    plt.show()

# Aplicar la función a todas las columnas numéricas del dataframe
columnas_cuantitativas = df.select_dtypes(include=['int64', 'float64']).columns
for columna in columnas_cuantitativas:
    analizar_variable_cuantitativa(df, columna)

## Análisis de datos cualitativos


In [ ]:
# Función para el análisis univariado de variables categóricas
def analizar_variable_categorica(df, columna):
    values_counts = df[columna].value_counts()
    moda = values_counts.idxmax()

    print(f"Análisis Univariado de la Variable '{columna}'")
    print(f"Frecuencia de las categorías:\n{values_counts}")
    print(f"Moda (Categoría más frecuente): {moda}")
    print("\n")

    # Visualización de la distribución
    plt.figure(figsize=(10, 6))
    sns.countplot(
        x=columna, 
        data=df[df[columna].isin(values_counts.index)], 
        order=values_counts.index, 
        palette='Set2'
    )
    plt.title(f'Distribución de {columna}')
    plt.xlabel(columna)
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45)
    plt.show()

# Columnas categóricas a excluir
columnas_a_excluir = ['TransactionDate']

columnas_categoricas = [col for col in df.select_dtypes(include=['object']).columns if col not in columnas_a_excluir]

for columna in columnas_categoricas:
    analizar_variable_categorica(df, columna)


## Verificar si hay patrones o relaciones presentes entre variables

## Verificar si hay valores nulos en el dataset
### Corrección de valores nulos presentes en el dataset

In [ ]:
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Procesamiento de datos
# Verificar si hay valores atípicos

In [ ]:
def generar_boxplot(df, columna):
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x=columna, palette='Set2')
    plt.title(f'Boxplot de {columna}')
    plt.xlabel(columna)
    plt.show()

# Aplicar la función a todas las columnas numéricas del dataframe
columnas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
for columna in columnas_numericas:
    generar_boxplot(df, columna)

- Para el tratamiento de datos atípicos lo que vamos a realizar es la imputación de ellos mediante el método intercuartílico, pero también vamos a generar un dataset sin realizar ningún tratamiento de los datos outlayers presentes en el dataset para realizar pruebas posteriormente en la aplicación de modelos de machine learning para ver cuanta diferencia hay entre ambos casos.

In [ ]:
df_clean = df.copy()

variable_objetivo = 'IsFraud'
df_sin_objetivo = df_clean.drop(columns=[variable_objetivo])

for col in df_sin_objetivo.select_dtypes(include=['int64', 'float64']).columns:
    # Calcula Q1 y Q3
    Q1 = df_sin_objetivo[col].quantile(0.25)
    Q3 = df_sin_objetivo[col].quantile(0.75)
    IQR = Q3 - Q1
    
    # Define los límites inferior y superior
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    # Imputa los valores atípicos por los límites correspondientes
    df_sin_objetivo[col] = np.where(df_sin_objetivo[col] < lower_limit, lower_limit, df_sin_objetivo[col])
    df_sin_objetivo[col] = np.where(df_sin_objetivo[col] > upper_limit, upper_limit, df_sin_objetivo[col])

# Reinserta la variable objetivo en el DataFrame limpio
df_clean = pd.concat([df_sin_objetivo, df_clean[[variable_objetivo]]], axis=1)

# Verificamos algunos valores antes y después de la limpieza
print("Datos antes de la limpieza:")
print(df.describe())
print("\nDatos después de la limpieza:")
print(df_clean.describe())

- Lo que realizamos aqui es explir nuestra varialbe objetivo la cual es IsFraud ya que uno de los princip[ales problemas es que al tenere pocos datos para uno de los valores lo clasificaba como outlayer y a la hora de apklciarldo estos datos se eliminaban, si bien son pocos datos lo que se iban eliminando siguen siendo importantes para la posterior balanceo de datos]

## Preparación de datos
### Vamos a realizar el tratamiento de datos pasos a realizar
### Convertir datos categóricos a numéricos para esto vamos a aplicar one hot encoder y label conder dependiendo a tipo de dato
### Además de convertir los datos categóricos en general para aplicar estos datos en modelos de machine learning, además de verificar el balanceo de datos y aplicación de técnicas para corregir variables con outlayers


In [ ]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Valores únicos en la columna '{column}':")
    print(unique_values)
    print("\n------------------------------------\n")

## LabelEncoder

### Variables Ninguna

## OneHotEncoder

### Variables: Location, TransactionType

In [ ]:
# Aplicar One-Hot Encoding
df = pd.get_dummies(df, columns=['Location'])
df = pd.get_dummies(df, columns=['TransactionType'])
# datos sin outlayers
df_clean = pd.get_dummies(df_clean, columns=['Location'])
df_clean = pd.get_dummies(df_clean, columns=['TransactionType'])

In [ ]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Valores únicos en la columna '{column}':")
    print(unique_values)
    print("\n------------------------------------\n")

### Verificacion sobre el balance de los datos de la variable IsFraud, para posterior tratamiento
- Con esto tenemos un problema, nuestra variable objetivo esta demasiado desbalanceada, po lo que tendremos que aplicar balanceo de datos ya bien sea con oversample (SMOTE) a la variable minoritaria o undersample a nuestra variable mayoritaria

In [ ]:
variable = 'IsFraud'
balance = df[variable].value_counts(normalize=True) * 100

print(f"Distribución de '{variable}':")
print(balance)

# Visualizar distribución con un gráfico de barras
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x=variable, order=balance.index, palette="viridis")
plt.title(f"Distribución de la variable '{variable}'")
plt.xlabel(variable)
plt.ylabel("Frecuencia")
plt.xticks(rotation=45)
plt.show()

### Aplicación de Class Weighting en el Modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Dividir en variables predictoras (X) y la variable objetivo (y)
X = df.drop(columns=["IsFraud", "TransactionDate", "TransactionType","Location"])
y = df["IsFraud"]

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Aplicación de class_weight para un modelo (ejemplo con RandomForest)
model_rf = RandomForestClassifier(class_weight="balanced", random_state=42)
model_rf.fit(X_train, y_train)

# Otro ejemplo con Logistic Regression
model_lr = LogisticRegression(class_weight="balanced", random_state=42)
model_lr.fit(X_train, y_train)

### Combination of Oversampling (con RandomOverSampler) y Class Weighting


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Crear un oversampler
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Crear un nuevo DataFrame con el dataset balanceado
df_balanced = pd.DataFrame(X_resampled, columns=X.columns)
df_balanced["IsFraud"] = y_resampled

# Calcular class_weight para el modelo
class_weights = compute_class_weight("balanced", classes=np.array([0, 1]), y=y_resampled)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Modelo con class_weight aplicado
model_balanced_rf = RandomForestClassifier(class_weight=class_weight_dict, random_state=42)
model_balanced_rf.fit(X_resampled, y_resampled)

print("Distribución después del balanceo:")
print(df_balanced["IsFraud"].value_counts())

In [ ]:
variable = 'IsFraud'
balance = df_balanced[variable].value_counts(normalize=True) * 100

print(f"Distribución de '{variable}':")
print(balance)

# Visualizar distribución con un gráfico de barras
plt.figure(figsize=(8, 5))
sns.countplot(data=df_balanced, x=variable, order=balance.index, palette="viridis")
plt.title(f"Distribución de la variable '{variable}'")
plt.xlabel(variable)
plt.ylabel("Frecuencia")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Calcular la matriz de correlación
corr_matrix = df_balanced.corr()

# Generar el mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Mapa de calor de la correlación entre variables')
plt.show()

In [ ]:
# Calcular la matriz de correlación
correlation_matrix = df_balanced.corr()

# Filtrar solo las correlaciones con la variable objetivo
target_corr = correlation_matrix['IsFraud'].sort_values(ascending=False)
print("Correlación de cada variable con 'Productivity_Score':\n", target_corr)

# Aplicación de modelos de machine learning

## Predicción con datos atípicos


In [ ]:
# Variables utilizadas
print("Variables utilizadas en el modelo:")
print(X.columns)

## Predicción sin datos atípicos


In [ ]:
from sklearn.model_selection import train_test_split

# Seleccionar características (features) y la variable objetivo (target)
X = df_balanced.drop(columns=['IsFraud'])
y = df_balanced['IsFraud']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Crear el modelo de Regresión Logística
model = LogisticRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy_RL = accuracy_score(y_test, y_pred)
matrix_RL = confusion_matrix(y_test, y_pred)
reporte_RL = classification_report(y_test, y_pred)

print(f'Precisión del modelo: {accuracy_RL:.2f}')
print('Matriz de confusión:')
print(matrix_RL)
print('Reporte de clasificación:')
print(reporte_RL)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

# Crear el modelo de Árbol de Decisión
model = DecisionTreeClassifier()

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy_AD = accuracy_score(y_test, y_pred)
matrix_AD = confusion_matrix(y_test, y_pred)
reporte_AD = classification_report(y_test, y_pred)

print(f'Precisión del modelo: {accuracy_AD:.2f}')
print('Matriz de confusión:')
print(matrix_AD)
print('Reporte de clasificación:')
print(reporte_AD)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Crear el modelo de Support Vector Machine
model = SVC()

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy_SVM = accuracy_score(y_test, y_pred)
matrix_SVM = confusion_matrix(y_test, y_pred)
reporte_SVM = classification_report(y_test, y_pred)

print(f'Precisión del modelo: {accuracy_SVM:.2f}')
print('Matriz de confusión:')
print(matrix_SVM)
print('Reporte de clasificación:')
print(reporte_SVM)